# **Customer Segmentation Analysis - RFM (Recency, Frequency & Monetary Value)**

**RFM**, also known as RFM analysis, is a type of customer segmentation and behavioral targeting used to help businesses rank and segment customers based on the recency, frequency, and monetary value of a transaction. RFM marketing can help marketers and small business owners determine their target audience to use their budget most effectively.

**Frequency**: Frequency is how often the customer makes purchases, which can help you identify repeat customers. For example, many clients make frequent repeat purchases within a set timeframe. Frequency is essential in determining the individuals most likely to continue shopping with your brand after their first initial purchase.

**Monetary value**: Monetary value refers to how much a customer spends within a given period. It's always important to consider because it can tell you a few things about consumer behavior. For example, you might find that customers with the highest monetary value don't purchase items as frequently as others but typically buy the most expensive products when they do.

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input" directory.

import time, warnings
import datetime as dt

#visualizations
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

## **Reading Data**

In [10]:
retail_df = pd.read_csv('/content/drive/MyDrive/BP3-ADA-US/data.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


## **Prepare Data**


In [13]:
# Count number of unique customers, rows per each country

country_count = retail_df.groupby('Country')['CustomerID'].agg(['nunique', 'count']).sort_values(by='nunique', ascending=False)
country_count.head()

,nunique,count
Country,,
United Kingdom,3950,361878
Germany,95,9495
France,87,8491
Spain,31,2533
Belgium,25,2069


In [15]:
retail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  object 
 7   Country      541909 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB


In [21]:
retail_df.shape

(397924, 8)

In [17]:
# Removing canceled orders from Quantity column which are minus values
retail_df = retail_df[retail_df['Quantity'] > 0]

In [20]:
# dataframe with unique value of customer ids and their scores
retail_df = retail_df.dropna(subset=['CustomerID'], how='all')

In [22]:
print("Summary....")
print(f"Total Number of Transactions: {retail_df.shape[0]}")
print(f"Total Number of Unique Customers: {retail_df['CustomerID'].nunique()}")
print(f"Percentage of Null Values: {round(retail_df.isnull().sum().sum()/retail_df.shape[0]*100, 2)}%")

Summary....
Total Number of Transactions: 397924
Total Number of Unique Customers: 4339
Percentage of Null Values: 0.0%


## **RFM analysis**

/per customer

* **Recency** (R): Days since last purchase
* **Frequency** (F): Total number of purchases
* **Monetary Value** (M): Total money spent

### **Recency**

In [23]:
# last available date in our dataset
retail_df['InvoiceDate'].max()

'9/9/2011 9:52'

In [24]:
retail_df['InvoiceDate'].min()

'1/10/2011 10:32'

In [25]:
now = dt.date(2011, 12, 9)
print(now)

2011-12-09


In [28]:
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom,2010-12-01
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom,2010-12-01


In [27]:
retail_df['date'] = pd.DatetimeIndex(retail_df['InvoiceDate']).date

In [31]:
# unique rows per each customer and find out their last purchase date for recency
recency_dataframe = retail_df.groupby(by = 'CustomerID', as_index = False)['date'].max()
recency_dataframe.columns = ['CustomerID', 'LastPurchaseDate']
recency_dataframe.head()

,CustomerID,LastPurchaseDate
0,12346,2011-01-18
1,12347,2011-12-07
2,12348,2011-09-25
3,12349,2011-11-21
4,12350,2011-02-02


In [32]:
# calculate recency values per each customer
recency_dataframe['Recency'] = recency_dataframe['LastPurchaseDate'].apply(lambda x: (now - x).days)

In [33]:
recency_dataframe.head()

,CustomerID,LastPurchaseDate,Recency
0,12346,2011-01-18,325
1,12347,2011-12-07,2
2,12348,2011-09-25,75
3,12349,2011-11-21,18
4,12350,2011-02-02,310


In [34]:
recency_dataframe = recency_dataframe.drop('LastPurchaseDate', axis = 1)

### **Frequency**

In [38]:
# frequency helps us to know how often customer purchased from the business
frequency_dataframe = retail_df.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep='first')
frequency_dataframe = retail_df.groupby(by = 'CustomerID', as_index=False)['InvoiceNo'].count()
frequency_dataframe.columns = ['CustomerID', 'Frequency']
frequency_dataframe.head()

,CustomerID,Frequency
0,12346,1
1,12347,182
2,12348,31
3,12349,73
4,12350,17


### **Monetary Value**

In [43]:
# How much money customer spent over time
# create column with the total spent per each customer

retail_df['TotalCost'] = retail_df['Quantity'] * retail_df['UnitPrice']
monetory_dataframe = retail_df.groupby(by = 'CustomerID', as_index = False).agg({'TotalCost' : 'sum'})
monetory_dataframe.columns = ['CustomerID', 'MonetaryValue']
monetory_dataframe.head()

,CustomerID,MonetaryValue
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


### **RFM Table**

In [45]:
# merge recency, frequency & monetary value dataframes all of them together
# inner join in SQL by primary & foreign keys by using ERD
merged_dataframe = recency_dataframe.merge(frequency_dataframe, on='CustomerID').merge(monetory_dataframe, on='CustomerID')
merged_dataframe.head()

,CustomerID,Recency,Frequency,MonetaryValue
0,12346,325,1,77183.60
1,12347,2,182,4310.00
2,12348,75,31,1797.24
3,12349,18,73,1757.55
4,12350,310,17,334.40


In [46]:
rfm_df = merged_dataframe.set_index('CustomerID')
rfm_df.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12346,325,1,77183.60
12347,2,182,4310.00
12348,75,31,1797.24
12349,18,73,1757.55
12350,310,17,334.40


[link to quantiles in stats](https://www.sigmamagic.com/blogs/what-are-quartiles/)

In [47]:
quantiles = rfm_df.quantile(q=[0.25, 0.5, 0.75])
quantiles

,Recency,Frequency,MonetaryValue
0.25,17.0,17.0,307.245
0.50,50.0,41.0,674.450
0.75,141.5,100.0,1661.640


In [48]:
quantiles.to_dict()

{'Recency': {0.25: 17.0, 0.5: 50.0, 0.75: 141.5},
 'Frequency': {0.25: 17.0, 0.5: 41.0, 0.75: 100.0},
 'MonetaryValue': {0.25: 307.245, 0.5: 674.45, 0.75: 1661.64}}

### **Create RFM Segments**

In [49]:
# user defined function to segment our values based on quantiles

def RScore(x, p, d):
  if x <= d[p][0.25]:
    return 4
  elif x <= d[p][0.50]:
    return 3
  elif x <= d[p][0.75]:
    return 2
  else:
    return 1

def FMScore(x, p, d):
  if x <= d[p][0.25]:
    return 1
  elif x <= d[p][0.50]:
    return 2
  elif x <= d[p][0.75]:
    return 3
  else:
    return 4

In [50]:
rfm_segmentation = rfm_df
rfm_segmentation['R'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency', quantiles,))
rfm_segmentation['F'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency', quantiles,))
rfm_segmentation['M'] = rfm_segmentation['MonetaryValue'].apply(FMScore, args=('MonetaryValue', quantiles,))

In [51]:
rfm_segmentation.head()

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12346,325,1,77183.60,1,1,4
12347,2,182,4310.00,4,4,4
12348,75,31,1797.24,2,2,4
12349,18,73,1757.55,3,3,4
12350,310,17,334.40,1,1,2


In [53]:
rfm_segmentation['RFMScore'] = rfm_segmentation['R'].map(str) + rfm_segmentation['F'].map(str) + rfm_segmentation['M'].map(str)
rfm_segmentation.head()

,Recency,Frequency,MonetaryValue,R,F,M,RFMScore
CustomerID,,,,,,,
12346,325,1,77183.60,1,1,4,114
12347,2,182,4310.00,4,4,4,444
12348,75,31,1797.24,2,2,4,224
12349,18,73,1757.55,3,3,4,334
12350,310,17,334.40,1,1,2,112


In [56]:
# BEST RECENCY SCORE = 4
# BEST FREQUENCU SCORE = 4
# BEST MV SCORE = 4

# Identify best customers for business

rfm_segmentation[rfm_segmentation['RFMScore'] == '444'].sort_values('MonetaryValue', ascending = False)

,Recency,Frequency,MonetaryValue,R,F,M,RFMScore
CustomerID,,,,,,,
14646,1,2080,280206.02,4,4,4,444
18102,0,431,259657.30,4,4,4,444
17450,8,337,194550.79,4,4,4,444
14911,1,5677,143825.06,4,4,4,444
14156,9,1400,117379.63,4,4,4,444
...,...,...,...,...,...,...,...
17813,14,371,1770.88,4,4,4,444
13184,14,181,1701.29,4,4,4,444
16775,10,158,1695.66,4,4,4,444
